<a href="https://colab.research.google.com/github/manuelapop/python_exercises/blob/main/final_scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



this is the semi gradient update exercise



question 1 chapter 9
Semi-Gradient Update
This problem presents a brief glimpse of the problems that can arise in off-policy learning with function approximation, through the concepts that have been introduced so far. If you would like a more detailed discussion on these issues, you may refer to Chapter 11. Let us now apply semi-gradient TD learning from Chapter 9 with batch updates (Section 6.3) to the following value-function approximation problem, based on a problem known as Baird's Counterexample:

In [2]:
import argparse
import sys
import numpy as np

NUM_STATES = 6
NUM_WEIGHTS = 7
TOP_STATES = list(range(5))
BOTTOM_STATE = 5
TRANSITIONS = [(i, BOTTOM_STATE) for i in TOP_STATES] + [(BOTTOM_STATE, BOTTOM_STATE)]
REWARD = 0.0

def phi(state: int):
    f = np.zeros(NUM_WEIGHTS)
    if state in TOP_STATES:
        f[0] = 1.0
        f[state + 1] = 2.0
    else:
        f[0] = 2.0
        f[6] = 1.0
    return f

def value(state, w):
    return float(phi(state) @ w)

def batch_td0_update(w, alpha, gamma):
    dw = np.zeros_like(w)
    for s, s_next in TRANSITIONS:
        delta = REWARD + gamma * value(s_next, w) - value(s, w)
        dw += alpha * delta * phi(s)
    return w + dw

def run_experiment(alpha, gamma, w0, num_batches):
    w = np.array(w0, dtype=float)
    print("Initial weights:", w)

    for b in range(1, num_batches + 1):
        print(f"\n=== Batch {b} ===")
        print(f"Bottom value before: {value(BOTTOM_STATE, w):.3f}")
        print(f"Top value before:    {value(TOP_STATES[0], w):.3f}")

        w = batch_td0_update(w, alpha, gamma)

        print("Updated weights:", np.round(w, 3))
        print(f"Bottom value after:  {value(BOTTOM_STATE, w):.3f}")
        print(f"Top value after:     {value(TOP_STATES[0], w):.3f}")

    print("\nFinal weights:", np.round(w, 3))

def parse_args():
    parser = argparse.ArgumentParser()

    parser.add_argument("--alpha", type=float, default=0.1)
    parser.add_argument("--gamma", type=float, default=0.95)
    parser.add_argument("--weights", type=float, nargs=7,
                        default=[1,1,1,1,1,1,5])
    parser.add_argument("--batches", type=int, default=1)

    # Fix for Jupyter/Colab:
    if "ipykernel" in sys.modules:
        return parser.parse_args([])
    else:
        return parser.parse_args()

if __name__ == "__main__":
    args = parse_args()
    run_experiment(args.alpha, args.gamma, args.weights, args.batches)


Initial weights: [1. 1. 1. 1. 1. 1. 5.]

=== Batch 1 ===
Bottom value before: 7.000
Top value before:    3.000
Updated weights: [2.755 1.73  1.73  1.73  1.73  1.73  4.965]
Bottom value after:  10.475
Top value after:     6.215

Final weights: [2.755 1.73  1.73  1.73  1.73  1.73  4.965]


question 2 chapter 10 Now we will calculate the values of State A and B for the MDP mentioned above. We put the problem description here again: 2 states and 3 states

In [4]:
import math

# ---------- Utility helpers ----------

def make_periodic_reward(period):
    """
    Given a list like [1, 0] or [1, 0, 0], return a function
    reward(t) that returns the reward at time step t (0-based)
    for that periodic sequence.
    """
    T = len(period)
    def reward_fn(t):
        return period[t % T]
    return reward_fn


def average_reward(reward_fn, horizon=10_000):
    """
    Time-average reward over a finite horizon, which approximates
    the infinite-horizon average reward.
    """
    total = 0.0
    for t in range(horizon):
        total += reward_fn(t)
    return total / horizon


def differential_value(reward_fn, avg_r, gamma=0.999, T=10_000):
    """
    Approximate v(s) = sum_{t>=0} gamma^t (R_{t+1} - avg_r)
    for a single starting state with reward sequence reward_fn(t).
    """
    v = 0.0
    g = 1.0
    for t in range(T):
        r = reward_fn(t)           # R_{t+1} (shifted, but deterministic)
        v += g * (r - avg_r)
        g *= gamma
    return v


# ---------- 2-state alternating example ----------

def two_state_example():
    print("=== 2-state alternating MDP ===")
    # State A: rewards 1,0,1,0,...
    # State B: rewards 0,1,0,1,...
    reward_A = make_periodic_reward([1, 0])
    reward_B = make_periodic_reward([0, 1])

    # Average reward should be 0.5
    r_bar_A = average_reward(reward_A)
    r_bar_B = average_reward(reward_B)
    r_bar = 0.5 * (r_bar_A + r_bar_B)  # both equal, so any is fine

    print(f"Estimated average reward r(pi) ≈ {r_bar:.4f}")

    # Differential values for gamma close to 1
    for gamma in [0.9, 0.99, 0.999, 0.9999]:
        vA = differential_value(reward_A, r_bar, gamma=gamma)
        vB = differential_value(reward_B, r_bar, gamma=gamma)
        print(f"gamma={gamma:>7}: v(A)≈{vA:.4f}, v(B)≈{vB:.4f}")

    print("Analytic values from theory: v(A)=+0.25, v(B)=-0.25\n")


# ---------- 3-state ring example L'hopital rule ----------

def three_state_example():
    print("=== 3-state ring MDP (A -> B -> C -> A; reward 1 on A) ===")

    # Reward when ARRIVING in A is 1, else 0.
    # We encode the *reward sequence* seen starting from each state.

    # Start at A: A->B (0), B->C (0), C->A (1), then repeat: [0,0,1]
    reward_A = make_periodic_reward([0, 0, 1])

    # Start at B: B->C (0), C->A (1), A->B (0): [0,1,0]
    reward_B = make_periodic_reward([0, 1, 0])

    # Start at C: C->A (1), A->B (0), B->C (0): [1,0,0]
    reward_C = make_periodic_reward([1, 0, 0])

    # Average reward (should be 1/3)
    # Any starting state gives the same long-run average, so we just use C
    r_bar = average_reward(reward_C)
    print(f"Estimated average reward r(pi) ≈ {r_bar:.4f}  (theory: 1/3 ≈ 0.3333)")

    # Differential values for gamma close to 1
    for gamma in [0.9, 0.99, 0.999, 0.9999]:
        vA = differential_value(reward_A, r_bar, gamma=gamma)
        vB = differential_value(reward_B, r_bar, gamma=gamma)
        vC = differential_value(reward_C, r_bar, gamma=gamma)
        print(
            f"gamma={gamma:>7}: "
            f"v(A)≈{vA:.4f}, v(B)≈{vB:.4f}, v(C)≈{vC:.4f}"
        )

    print("Analytic values from theory: v(A)=-1/3, v(B)=0, v(C)=+1/3\n")


# ---------- Main ----------

if __name__ == "__main__":
    two_state_example()
    three_state_example()


=== 2-state alternating MDP ===
Estimated average reward r(pi) ≈ 0.5000
gamma=    0.9: v(A)≈0.2632, v(B)≈-0.2632
gamma=   0.99: v(A)≈0.2513, v(B)≈-0.2513
gamma=  0.999: v(A)≈0.2501, v(B)≈-0.2501
gamma= 0.9999: v(A)≈0.1580, v(B)≈-0.1580
Analytic values from theory: v(A)=+0.25, v(B)=-0.25

=== 3-state ring MDP (A -> B -> C -> A; reward 1 on A) ===
Estimated average reward r(pi) ≈ 0.3334  (theory: 1/3 ≈ 0.3333)
gamma=    0.9: v(A)≈-0.3451, v(B)≈-0.0130, v(C)≈0.3560
gamma=   0.99: v(A)≈-0.3411, v(B)≈-0.0078, v(C)≈0.3289
gamma=  0.999: v(A)≈-0.4001, v(B)≈-0.0668, v(C)≈0.2669
gamma= 0.9999: v(A)≈-0.7548, v(B)≈-0.5441, v(C)≈0.0346
Analytic values from theory: v(A)=-1/3, v(B)=0, v(C)=+1/3



question 3 chapter 12 Consider the following episode in an MRP:

In [7]:
from typing import List

def n_step_return(
    rewards: List[float],
    values: List[float],
    gamma: float,
    t: int,
    n: int
) -> float:
    """
    chapter12
    Compute the n-step return G_{t:t+n} for an episode.
    Calculate the forward view-return for state 0, up to 6 decimal places.

    rewards: [R1, R2, ..., RT]
    values:  [V(S0), V(S1), ..., V(ST)]  # length = len(rewards) + 1
    gamma: discount factor
    t: starting time index (0-based, corresponds to S_t)
    n: number of steps for the n-step return

    Returns G_{t:t+n}.
    """
    T = len(rewards)  # last reward index is T-1 (for R_T)

    # Make sure n does not step beyond the episode
    n = min(n, T - t)

    G = 0.0
    # First sum the discounted rewards
    for k in range(1, n + 1):
        G += (gamma ** (k - 1)) * rewards[t + k - 1]  # R_{t+k}

    # Bootstrap with value at S_{t+n}, if it exists
    if t + n <= T:
        G += (gamma ** n) * values[t + n]

    return G


def full_return(
    rewards: List[float],
    gamma: float,
    t: int
) -> float:
    """
    Monte Carlo return G_t from time t to the end of the episode.
    """
    G = 0.0
    power = 0
    for k in range(t, len(rewards)):
        G += (gamma ** power) * rewards[k]
        power += 1
    return G


def lambda_return(
    rewards: List[float],
    values: List[float],
    gamma: float,
    lam: float,
    t: int
) -> float:
    """
    Forward-view lambda-return G_t^λ for state S_t.

    rewards: [R1, ..., RT]
    values:  [V(S0), ..., V(ST)]
    gamma: discount factor
    lam: trace-decay parameter λ
    t: starting time index (0-based)
    """
    T = len(rewards)
    max_n = T - t  # maximum steps from t until the end

    # Last term: full return from t
    G_MC = full_return(rewards, gamma, t)

    # Sum of weighted n-step returns
    weighted_sum = 0.0
    for n in range(1, max_n):  # 1 ... (T - t - 1)
        G_n = n_step_return(rewards, values, gamma, t, n)
        weighted_sum += (lam ** (n - 1)) * G_n

    if max_n > 0:
        G_lambda = (1 - lam) * weighted_sum + (lam ** (max_n - 1)) * G_MC
    else:
        # No future rewards; λ-return is just V(S_t) by convention
        G_lambda = values[t]

    return G_lambda
if __name__ == "__main__":
    # From your screenshot example
    rewards = [1, 0, 1, 0, 1]  # R1..R6
    values  = [0.0, 0.1, 0.2, 0.3, 0.4]  # V(S0..S5)
    gamma = 0.9
    lam = 0.5
    t = 0  # state S0

    # Example: 1-step and 3-step returns from S0
    print("G_{0:1} =", n_step_return(rewards, values, gamma, t, 1))
    print("G_{0:3} =", n_step_return(rewards, values, gamma, t, 3))

    # Lambda-return from S0
    print("G_0^lambda =", lambda_return(rewards, values, gamma, lam, t))


G_{0:1} = 1.09
G_{0:3} = 2.0287
G_0^lambda = 1.37274625


question 4 chapter 12 Now suppose we were computing the n-step truncated TD(
), for horizon
.

In [8]:
from typing import List

def n_step_return(rewards: List[float],
                  values: List[float],
                  gamma: float,
                  t: int,
                  n: int) -> float:
    """
    G_{t:t+n}: n-step bootstrapped return starting at time t.
    rewards: R_1, ..., R_T   (length T)
    values : V(S_0), ..., V(S_T)  (state values)
    """
    T = len(rewards)
    n = min(n, T - t)          # cannot go past the end
    G = 0.0
    for k in range(1, n + 1):
        G += (gamma ** (k - 1)) * rewards[t + k - 1]

    # bootstrap if we haven't reached the end
    if t + n < T:
        G += (gamma ** n) * values[t + n]

    return G


def lambda_return_full(rewards: List[float],
                       values: List[float],
                       gamma: float,
                       lam: float,
                       t: int) -> float:
    """
    Full forward-view lambda-return (no truncation).
    """
    T = len(rewards)

    # full Monte Carlo return G_t
    G_full = 0.0
    for k in range(1, T - t + 1):
        G_full += (gamma ** (k - 1)) * rewards[t + k - 1]

    s = 0.0
    for n in range(1, T - t):
        G_n = n_step_return(rewards, values, gamma, t, n)
        s += (lam ** (n - 1)) * G_n

    return (1 - lam) * s + (lam ** (T - t - 1)) * G_full


def lambda_return_truncated(rewards: List[float],
                            values: List[float],
                            gamma: float,
                            lam: float,
                            t: int,
                            h: int) -> float:
    """
    n-step truncated TD(lambda) target with horizon h.
    G_t^{lambda,h} = (1-lam) * sum_{n=1}^{h-1} lam^{n-1} G_{t:t+n}
                     + lam^{h-1} * G_{t:t+h}
    """
    T = len(rewards)
    h = min(h, T - t)    # horizon can't exceed remaining length

    s = 0.0
    for n in range(1, h):
        G_n = n_step_return(rewards, values, gamma, t, n)
        s += (lam ** (n - 1)) * G_n

    G_h = n_step_return(rewards, values, gamma, t, h)
    return (1 - lam) * s + (lam ** (h - 1)) * G_h
rewards = [1, 0, 1, 0, 1]        # R1..R5
values  = [0, 0.1, 0.2, 0.3, 0.4]  # V(S0)..V(S4)
gamma   = 0.9
lam     = 0.5
t       = 0
h       = 3

print("Truncated λ-return (h=3) from S0:",
      lambda_return_truncated(rewards, values, gamma, lam, t, h))


Truncated λ-return (h=3) from S0: 1.342675


question 5 chapter 12 We know that we can express the return
 in terms of the TD errors at steps
,
.

In [9]:
"""
lambda_return_from_td_errors

This function computes the forward-view λ-return G_t^λ for a *single* time step t
using TD errors (δ_t, δ_{t+1}, …, δ_{T-1}).

Formula:
    G_t^λ = v_hat(S_t, w) + Σ_{k=t}^{T-1} (γλ)^{k-t} δ_k

Inputs
------
td_errors : sequence of floats
    TD errors from time t onward: [δ_t, δ_{t+1}, ..., δ_{T-1}]
gamma : float
    Discount factor γ (0 ≤ γ ≤ 1)
lam : float
    Trace decay parameter λ (0 ≤ λ ≤ 1)
v_hat_st : float, optional (default=0.0)
    Current value estimate for state S_t, v̂(S_t, w)

Output
------
G_lambda : float
    The λ-return G_t^λ for the starting time t corresponding to td_errors[0]
"""

def lambda_return_from_td_errors(td_errors, gamma, lam, v_hat_st=0.0):
    """Compute G_t^λ from TD errors starting at time t."""
    gl = v_hat_st
    gl_mult = 1.0            # will hold (γλ)^{k-t}
    gl_factor = gamma * lam  # γλ

    for delta_k in td_errors:
        gl += gl_mult * delta_k
        gl_mult *= gl_factor

    return gl


# ------------------ Example from your screenshot ------------------ #
if __name__ == "__main__":
    td_errors = [1.1, 0.4, 0.8, -0.5, 0.7]  # δ_t ... δ_{t+4}
    gamma = 0.9
    lam = 0.5
    v_hat_st = 0.0

    G_lambda = lambda_return_from_td_errors(td_errors, gamma, lam, v_hat_st)
    print("G_t^λ =", G_lambda)      # -> 1.425141875...
    print("Rounded to 6 decimals:", round(G_lambda, 6))  # 1.425142


G_t^λ = 1.4251418750000002
Rounded to 6 decimals: 1.425142


Question 6: this is for the last question in the prcatice final examp and also question in the homework on eligibility traces chapter 12 Backward View

In [1]:
"""
Backward-view eligibility traces and TD(λ) for tabular value functions.

This script:

  • Implements the backward-view eligibility trace update
        z ← γ λ z + ∇v̂(S_t, w)
    for a tabular / one-hot feature representation.

  • Uses it to reproduce the example from the gridworld problem where the
    eligibility trace after visiting states 5 → 4 → 3 has z[5] ≈ 0.0729.

  • Implements tabular TD(λ) with backward-view traces and applies it to
    the gridworld trajectory 5 → 4 → 3 → 2 → 1 → G with given rewards,
    returning the final values V(1)…V(5).
"""

from typing import List
import numpy as np


# ---------------------------------------------------------------------
# 1. Eligibility traces (backward view, tabular / one-hot features)
# ---------------------------------------------------------------------
def update_eligibility_trace(
    trajectory: List[int],
    num_states: int,
    gamma: float,
    lam: float,
) -> np.ndarray:
    """
    Update eligibility traces for a given sequence of visited states.

    Args
    ----
    trajectory : list[int]
        Sequence of visited state indices in the order they are processed.
        States are assumed to be integers in [0, num_states - 1].
    num_states : int
        Total number of distinct states (size of the tabular feature vector).
    gamma : float
        Discount factor γ.
    lam : float
        Trace decay parameter λ.

    Returns
    -------
    z : np.ndarray, shape (num_states,)
        Eligibility-trace vector after processing the whole trajectory.
    """
    z = np.zeros(num_states, dtype=float)
    gl = gamma * lam

    for s in trajectory:
        # Decay previous trace
        z *= gl
        # Add gradient for current state (one-hot feature for state s)
        z[s] += 1.0

    return z


# ---------------------------------------------------------------------
# 2. TD(λ) with backward-view eligibility traces (tabular case)
# ---------------------------------------------------------------------
def td_lambda_tabular(
    states: List[int],
    rewards: List[float],
    num_states: int,
    gamma: float,
    lam: float,
    alpha: float,
    v_init: np.ndarray,
) -> np.ndarray:
    """
    One episode of tabular TD(λ) with backward-view eligibility traces.

    Args
    ----
    states : list[int]
        State sequence S_0, …, S_T (length T+1).
    rewards : list[float]
        Rewards r_{t+1} for transitions S_t → S_{t+1} (length T).
    num_states : int
        Number of distinct states.
    gamma : float
        Discount factor γ.
    lam : float
        Trace decay parameter λ.
    alpha : float
        Learning rate α.
    v_init : np.ndarray
        Initial value function, shape (num_states,).

    Returns
    -------
    v : np.ndarray
        Value function after processing the entire episode.
    """
    v = v_init.copy()
    z = np.zeros(num_states, dtype=float)

    for t in range(len(rewards)):
        s = states[t]
        s_next = states[t + 1]
        r = rewards[t]

        # TD error δ_t = r_{t+1} + γ V(S_{t+1}) − V(S_t)
        delta = r + gamma * v[s_next] - v[s]

        # Eligibility trace update: z_t = γ λ z_{t−1} + x(S_t)
        z *= gamma * lam
        z[s] += 1.0

        # Weight / value update
        v += alpha * delta * z

    return v


# ---------------------------------------------------------------------
# 3. Run both examples
# ---------------------------------------------------------------------
if __name__ == "__main__":
    # ===== Example 1: eligibility trace after states 5 → 4 → 3 =====
    # We consider 6 states: [G, 1, 2, 3, 4, 5] with indices 0..5.
    # Here we only care about the trace components.
    trajectory = [5, 4, 3]  # indices of visited states

    num_states = 6
    gamma = 0.9
    lam = 0.3

    z = update_eligibility_trace(trajectory, num_states, gamma, lam)

    print("=== Example 1: eligibility traces ===")
    print("Eligibility trace vector z after processing states 5 → 4 → 3:")
    print(z)
    print(f"Component of z associated with state 5 (index 5): {z[5]:.6f}")
    # Expected ≈ 0.0729

    # ===== Example 2: TD(λ) values after full episode 5 → 4 → 3 → 2 → 1 → G =====
    #
    # State indices: 0 = G, 1 = state 1, …, 5 = state 5
    # Episode: 5 → 4 → 3 → 2 → 1 → G
    # Rewards for transitions:
    #   5 → 4 :  0
    #   4 → 3 : -0.1
    #   3 → 2 : -0.1
    #   2 → 1 :  0
    #   1 → G : +1
    states = [5, 4, 3, 2, 1, 0]
    rewards = [0.0, -0.1, -0.1, 0.0, 1.0]

    gamma = 0.9
    lam = 0.5
    alpha = 0.1

    # Initial values:
    # V(1) = 0, V(2) = 0, V(3) = -1, V(4) = 0, V(5) = 0, V(G) = 0
    v_init = np.zeros(num_states, dtype=float)
    v_init[3] = -1.0  # state 3

    v_final = td_lambda_tabular(
        states=states,
        rewards=rewards,
        num_states=num_states,
        gamma=gamma,
        lam=lam,
        alpha=alpha,
        v_init=v_init,
    )

    print("\n=== Example 2: TD(λ) final values after full episode ===")
    for s in range(1, 6):
        print(f"V({s}) = {v_final[s]:.3f}")
    # To three decimals this should give approximately:
    # V(1) ≈  0.100
    # V(2) ≈  0.045
    # V(3) ≈ -0.890
    # V(4) ≈ -0.050
    # V(5) ≈ -0.023


=== Example 1: eligibility traces ===
Eligibility trace vector z after processing states 5 → 4 → 3:
[0.     0.     0.     1.     0.27   0.0729]
Component of z associated with state 5 (index 5): 0.072900

=== Example 2: TD(λ) final values after full episode ===
V(1) = 0.100
V(2) = 0.045
V(3) = -0.890
V(4) = -0.050
V(5) = -0.023


Question 7 chapter 12 calculating What is the TD-error
 for the transition from state 5 to state 4?

In [11]:
"""
td_error.py

Small helper script for Temporal-Difference (TD) learning.

This file defines a function `td_error` that computes the TD error

    δ_t = R_{t+1} + γ * V(S_{t+1}) - V(S_t)

given:
    - current state's value V(S_t)
    - next state's value V(S_{t+1})
    - reward R_{t+1} from the transition
    - discount factor γ

At the bottom there is a worked example using your numbers:
transition from state 5 to state 4 with:
    V(5) = 30, V(4) = 25, R = 0, γ = 0.9
which should give δ = -7.5
"""


def td_error(v_current: float, v_next: float, reward: float, gamma: float) -> float:
    """
    Compute the TD error for a single transition.

    Parameters
    ----------
    v_current : float
        V(S_t) - estimated value of the current state.
    v_next : float
        V(S_{t+1}) - estimated value of the next state.
    reward : float
        R_{t+1} - reward received when transitioning from S_t to S_{t+1}.
    gamma : float
        Discount factor γ (0 <= γ <= 1).

    Returns
    -------
    float
        The TD error δ_t.
    """
    return reward + gamma * v_next - v_current


if __name__ == "__main__":
    # --- Example: transition from state 5 -> state 4 ---
    V_5 = 30.0   # V(5)
    V_4 = 25.0   # V(4)
    R_5_to_4 = 0.0
    gamma = 0.9

    delta = td_error(V_5, V_4, R_5_to_4, gamma)
    print(f"TD error for transition 5 -> 4: δ = {delta}")   # expected: -7.5


TD error for transition 5 -> 4: δ = -7.5


Question 8 chapter 12 What will be the new value of state
 (i.e.
) if we make updates according to TD(
) at each step with
, at the end of the episode?

In [12]:
"""
TD(lambda) backward-view weight update for a tabular value function
matching the homework example.

- States: [goal, 1, 2, 3, 4, 5]
- One-hot features (tabular)
- γ = 0.9
- λ = 0.3
- α = 0.1
- Trajectory: 5 → 4 → 3 → 2 → 1 → goal
- Rewards: 0 on all steps, except last step reward = 1
"""

import numpy as np

# Discount, trace decay, learning rate
gamma = 0.9
lam = 0.3
alpha = 0.1

# Initial values (index = state number)
# Index 0 = goal state
w = np.array([0.0, 10.0, 15.0, 20.0, 25.0, 30.0], dtype=float)

# Eligibility trace vector (same size as w)
z = np.zeros_like(w)

# Trajectory states (excluding the final terminal transition)
trajectory = [5, 4, 3, 2, 1, 0]   # ending at goal (state 0)
rewards = [0, 0, 0, 0, 1]        # reward for each transition

print("Initial w =", w)
print("\n===== TD(lambda) Updates =====")

for t in range(len(rewards)):
    s = trajectory[t]       # current state
    s_next = trajectory[t+1]  # next state
    r = rewards[t]

    # --- Eligibility trace update ---
    # z = γλz + ∇v(s)
    # For tabular: ∇v(s) is 1 for component s
    z = gamma * lam * z
    z[s] += 1.0

    # --- TD error ---
    delta = r + gamma * w[s_next] - w[s]

    # --- Weight update ---
    w = w + alpha * delta * z

    print(f"\nState {s} → {s_next}")
    print("z after update:", np.round(z, 6))
    print("δ:", round(delta, 6))
    print("w after update:", np.round(w, 6))

print("\n===== Final Value of State 5 =====")
print("V(5) =", round(w[5], 4))


Initial w = [ 0. 10. 15. 20. 25. 30.]

===== TD(lambda) Updates =====

State 5 → 4
z after update: [0. 0. 0. 0. 0. 1.]
δ: -7.5
w after update: [ 0.   10.   15.   20.   25.   29.25]

State 4 → 3
z after update: [0.   0.   0.   0.   1.   0.27]
δ: -7.0
w after update: [ 0.    10.    15.    20.    24.3   29.061]

State 3 → 2
z after update: [0.     0.     0.     1.     0.27   0.0729]
δ: -6.5
w after update: [ 0.       10.       15.       19.35     24.1245   29.013615]

State 2 → 1
z after update: [0.       0.       1.       0.27     0.0729   0.019683]
δ: -6.0
w after update: [ 0.       10.       14.4      19.188    24.08076  29.001805]

State 1 → 0
z after update: [0.       1.       0.27     0.0729   0.019683 0.005314]
δ: -9.0
w after update: [ 0.        9.1      14.157    19.12239  24.063045 28.997022]

===== Final Value of State 5 =====
V(5) = 28.997


Question 9 chapter 12
Just as the return can be written recursively in terms of the first reward and the return starting one-step later (i.e.
), we can also get a recursive form for
 - return.

In [13]:
"""
lambda_return_recursive.py

Compute the forward-view lambda-return G_t^λ using its recursive form:

    G_t^λ = R_{t+1} + γ [ λ G_{t+1}^λ + (1 - λ) v_hat(S_{t+1}) ]

This is the λ-return analogue of the recursive Bellman-style equation
G_t = R_{t+1} + γ G_{t+1}, and is used in forward-view TD(λ) analysis.
"""

def lambda_return_recursive(R_tp1, gamma, lam, G_lambda_tp1, v_next):
    """
    Compute the recursive λ-return G_t^λ.

    Parameters
    ----------
    R_tp1 : float
        Immediate reward R_{t+1}.
    gamma : float
        Discount factor γ.
    lam : float
        Trace-decay parameter λ.
    G_lambda_tp1 : float
        λ-return from the next time step, G_{t+1}^λ.
    v_next : float
        Bootstrapped value estimate at the next state, v_hat(S_{t+1}).

    Returns
    -------
    float
        The λ-return G_t^λ.
    """
    return R_tp1 + gamma * (lam * G_lambda_tp1 + (1 - lam) * v_next)


if __name__ == "__main__":
    # Example with the values from the homework screenshot
    R_tp1 = 10.0
    gamma = 0.9
    lam = 0.5
    G_lambda_tp1 = 0.7
    v_next = 9.0

    G_lambda_t = lambda_return_recursive(R_tp1, gamma, lam, G_lambda_tp1, v_next)
    print(f"G_lambda_t = {G_lambda_t:.4f}")  # -> 14.3650 (≈ 14.37)


G_lambda_t = 14.3650


Question 10 chapter 9 Now suppose we are using radial basis functions as our feature representation ( Use the equation on page 221 in Section 9.5.5 of the textbook. ).

In [1]:
import math

def rbf(state, center, sigma=1.0):
    """
    Radial Basis Function feature.
    state  = (x, y)   current state coordinates
    center = (cx, cy) center of this RBF
    sigma  = width parameter (here sigma = 1.0)

    rbf(s, c) = exp( - ||s - c||^2 / (2 * sigma^2) )
    """
    x, y = state
    cx, cy = center
    dist_sq = (x - cx)**2 + (y - cy)**2
    return math.exp(-dist_sq / (2.0 * sigma**2))


def value_approx(state, centers, weight=0.2, sigma=1.0):
    """
    Computes v(state) = sum_i w_i * rbf(state, c_i)
    Here, all weights w_i are the same (= weight).
    """
    total = 0.0
    for c in centers:
        total += rbf(state, c, sigma)
    return weight * total


if __name__ == "__main__":
    # Current state (yellow R)
    state = (3, 2)

    # Centers of the 5 RBFs (black dots)
    centers = [
        (1, 1),
        (1, 2),
        (2, 1),
        (2, 3),
        (3, 3),
    ]

    v = value_approx(state, centers, weight=0.2, sigma=1.0)
    print("Value approximation at state (3,2):", v)
    # If you want it rounded:
    print("Rounded to 2 decimals:", round(v, 2))


Value approximation at state (3,2): 0.31194196478320596
Rounded to 2 decimals: 0.31


Question 11 Chapter 9 Now if we use a fourier basis as the feature representation, what would be the approximation of the state value at the current location?

In [3]:
import math
from itertools import product
from typing import Sequence, List, Tuple


def dot(a: Sequence[float], b: Sequence[float]) -> float:
    """Simple dot product between two vectors."""
    return sum(x * y for x, y in zip(a, b))


def fourier_feature(state: Sequence[float], c: Sequence[float]) -> float:
    r"""
    Single Fourier-basis feature:
        x_i(s) = cos(pi * s^T c_i)
    where:
        state s  = (x, y, ...)         current state (normalized)
        c        = coefficient vector for this feature
    """
    return math.cos(math.pi * dot(state, c))


def fourier_value(
    state: Sequence[float],
    coeffs: Sequence[Sequence[float]],
    weights: Sequence[float],
) -> float:
    """
    Value approximation using a (possibly custom) set of Fourier-basis features.

        v_hat(s) = sum_i w_i * cos(pi * s^T c_i)

    Args
    ----
    state   : current state vector
    coeffs  : list of coefficient vectors c_i
    weights : list of weights w_i (same length as coeffs)
    """
    assert len(coeffs) == len(weights), "coeffs and weights must have same length"
    total = 0.0
    for c, w in zip(coeffs, weights):
        total += w * fourier_feature(state, c)
    return total


def generate_fourier_coeffs(order: int, dim: int) -> List[Tuple[int, ...]]:
    """
    Generate all Fourier coefficient vectors for a given max order.

    For example, order=1, dim=2 gives:
        (0,0), (0,1), (1,0), (1,1)

    This is useful if you want an n-th order Fourier basis automatically.
    """
    return list(product(range(order + 1), repeat=dim))
if __name__ == "__main__":
    # Current state (x, y)
    state = (3.0, 2.0)

    # Coefficient vectors c^0, c^1, c^2, c^3
    coeffs = [
        (0.0, 1.0),   # c^0
        (1.0, 0.0),   # c^1
        (0.2, 0.8),   # c^2
        (0.8, 0.2),   # c^3
    ]

    # Weights w_0, ..., w_3
    weights = [0.6, 0.3, 0.5, 0.3]

    v = fourier_value(state, coeffs, weights)
    print("Fourier value at state (3,2):", v)
    print("Rounded to 2 decimals:", round(v, 2))


Fourier value at state (3,2): 0.4618033988749894
Rounded to 2 decimals: 0.46


question 12, chapter 2 gredy action

In [3]:
def epsilon_definite_steps(actions, rewards, k=None):
    """
    Given sequences of actions and rewards for an epsilon-greedy bandit,
    compute at which timesteps the epsilon case MUST have occurred.

    We assume:
      - sample-average action-value estimates
      - initial Q_1(a) = 0 for all actions
      - ties in Q are broken arbitrarily, so ANY action in the argmax(Q)
        set could have been chosen greedily.

    Args
    ----
    actions : list[int]
        A_t sequence (1-based action indices).
    rewards : list[float]
        R_t sequence (same length as actions).
    k : int or None
        Number of arms. If None, uses max(actions).

    Returns
    -------
    definite_eps_steps : list[int]
        Timesteps t (1-based) at which the chosen action was NOT greedy,
        so epsilon exploration definitely happened.
    """
    import numpy as np

    assert len(actions) == len(rewards), "actions and rewards must match in length"

    if k is None:
        k = max(actions)

    # Tracking statistics
    Q = np.zeros(k)          # current Q-values
    counts = np.zeros(k)     # how many times each action has been taken
    sums = np.zeros(k)       # sum of rewards for each action

    definite_eps_steps = []

    print("=== Epsilon-definite analysis ===\n")
    for t, (a, r) in enumerate(zip(actions, rewards), start=1):
        a_idx = a - 1  # convert to 0-based index

        # ---- BEFORE updating with (a, r), decide if this step MUST be epsilon ----
        max_Q = np.max(Q)
        greedy_actions = [i + 1 for i, q in enumerate(Q) if q == max_Q]

        is_definitely_eps = (a not in greedy_actions)
        if is_definitely_eps:
            definite_eps_steps.append(t)

        print(f"Step {t}:")
        print(f"  Q before step = {Q}")
        print(f"  Greedy action(s) before step = {greedy_actions}")
        print(f"  Chosen action = {a}")
        print(f"  -> definitely epsilon? {is_definitely_eps}\n")

        # ---- Now update Q with the observed reward ----
        counts[a_idx] += 1
        sums[a_idx] += r
        Q[a_idx] = sums[a_idx] / counts[a_idx]

    print("Final Q-values:", Q)
    print("Timesteps where epsilon definitely occurred:", definite_eps_steps)
    return definite_eps_steps


# ------------------------------------------------------------------
# Example from your question
# ------------------------------------------------------------------
actions = [1, 2, 2, 2, 3]
rewards = [-1, 1, -2, 2, 0]

epsilon_definite_steps(actions, rewards)


=== Epsilon-definite analysis ===

Step 1:
  Q before step = [0. 0. 0.]
  Greedy action(s) before step = [1, 2, 3]
  Chosen action = 1
  -> definitely epsilon? False

Step 2:
  Q before step = [-1.  0.  0.]
  Greedy action(s) before step = [2, 3]
  Chosen action = 2
  -> definitely epsilon? False

Step 3:
  Q before step = [-1.  1.  0.]
  Greedy action(s) before step = [2]
  Chosen action = 2
  -> definitely epsilon? False

Step 4:
  Q before step = [-1.  -0.5  0. ]
  Greedy action(s) before step = [3]
  Chosen action = 2
  -> definitely epsilon? True

Step 5:
  Q before step = [-1.          0.33333333  0.        ]
  Greedy action(s) before step = [2]
  Chosen action = 3
  -> definitely epsilon? True

Final Q-values: [-1.          0.33333333  0.        ]
Timesteps where epsilon definitely occurred: [4, 5]


[4, 5]

question 13 - Chapter 3

In [4]:
def compute_returns(gamma, rewards):
    """
    Compute returns G_0, ..., G_n for a reward sequence with discount gamma.

    Args
    ----
    gamma : float
        Discount factor (0 <= gamma <= 1).
    rewards : list[float]
        Rewards [R1, R2, ..., Rn].

    Returns
    -------
    G : list[float]
        Returns [G_0, G_1, ..., G_n], with G_n = 0
        (assuming all rewards after R_n are zero).
    """

    n = len(rewards)
    # We store G_0 ... G_n (so length n+1)
    G = [0.0] * (n + 1)

    # Work backwards: G_n is 0 by assumption, then fill G_{n-1}, ..., G_0
    for t in reversed(range(n)):
        G[t] = rewards[t] + gamma * G[t + 1]

    return G


# -------------------------------------------------------------------
# Example from your screenshot
# gamma = 0.5
# R1 = -1, R2 = 2, R3 = 6, R4 = 3, R5 = 2
# -------------------------------------------------------------------
if __name__ == "__main__":
    gamma = 0.5
    rewards = [-1, 2, 6, 3, 2]

    G = compute_returns(gamma, rewards)

    # Pretty print: G_0 ... G_5
    for t, g in enumerate(G):
        print(f"G_{t} = {g}")


G_0 = 2.0
G_1 = 6.0
G_2 = 8.0
G_3 = 4.0
G_4 = 2.0
G_5 = 0.0
